# Atividade 1 - Etapa 2 - Identificar os períodos de deglutição nos arquivos válidos

In [2]:
import pandas as pd

df = pd.read_csv('arquivos.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,a00001.mp3,0.619257,0.591173,0.588934,0.576050,0.569039,0.558396,0.591615,0.588218,0.560607,...,-12.914958,-8.228361,-10.005320,-9.044372,-9.907545,-8.394353,-9.334880,-6.777099,-8.923019,-6.725427
1,a00002.mp3,0.579076,0.561442,0.574778,0.592735,0.597341,0.586346,0.579085,0.572200,0.559282,...,-4.921903,-4.178082,-4.070145,-4.082938,-4.426908,-4.124296,-4.116983,-3.228512,-2.619876,-1.892528
2,a00003.mp3,0.659991,0.666785,0.614244,0.595386,0.595084,0.606930,0.569941,0.548508,0.509913,...,-12.366119,-6.802912,-8.757007,-7.462750,-9.101726,-7.317169,-9.505013,-6.167764,-7.143456,-4.136239
3,a00004.mp3,0.595120,0.439680,0.370019,0.396822,0.355503,0.330981,0.379730,0.433511,0.469010,...,-1.225749,4.526946,-1.537243,0.773656,-1.744216,1.602321,-3.417079,0.585987,-1.309876,2.340520
4,a00005.mp3,0.600767,0.576361,0.621725,0.593066,0.625016,0.594525,0.563292,0.547832,0.565713,...,-11.525387,-6.937714,-9.290621,-9.333829,-8.506713,-6.808221,-8.480545,-6.727121,-6.899271,-4.344142


## Detector de Anomalias

In [14]:
# Anomaly Novelty Detection
from sklearn.ensemble import IsolationForest
import numpy as np


X = df[df.columns[1:]].values

clf = IsolationForest(random_state=42)
y = clf.fit_predict(X)
score = clf.score_samples(X)

df['score'] = score
np.unique(y, return_counts=True)

(array([-1,  1]), array([ 43, 447], dtype=int64))

In [15]:
from IPython.display import Image, Audio, display
from ipywidgets import widgets

blocos = []
for i, row in df.sort_values('score').head(5).iterrows():
    nome = row["0"].split(".")[0]
    out = widgets.Output()
    with out:
        display(Image(f'dataset/Charts/{nome}.png'))
        display('Anormal')
        display(Audio(f'dataset/MP3/{row["0"]}'))
    blocos.append(out)
for i, row in df.sort_values('score', ascending=True).head(5).iterrows():
    out = widgets.Output()
    with out:
        display(Image(f'dataset/Charts/{nome}.png'))
        display('Normal')
        display(Audio(f'dataset/MP3/{row["0"]}'))
    blocos.append(out)
widgets.HBox(blocos)